In [1]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.1/618.1 KB 6.1 MB/s eta 0:00:002.9 MB/s eta 0:00:01
  Using cached scipy-1.11.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.3 MB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached torchvision-0.15.2-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached opencv_python-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.7 MB)
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 42.2 MB/s eta 0:00:00m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.7 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

# Loading YOLO models
detect_model = YOLO("Detect-20230921T094350Z-001/Detect/best.pt")
pattern_model = YOLO("Pattern-20230921T094355Z-001/Pattern/best.pt")
color_model = YOLO("Color-20230921T094349Z-001/Color/best.pt")


test = os.path.join('.', 'test')
video_path = os.path.join(test, 'test/WhatsApp Video 2023-09-20 at 6.00.02 PM.mp4')

# Open the video and get video properties
cap = cv2.VideoCapture(video_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
ret, frame = cap.read()
H, W, _ = frame.shape

# Get the user input for the second to start processing
desired_second = float(input("Enter the second to start processing: "))


start_frame = int(desired_second * fps)

threshold = 0.5

class_names = {
    0: 'sleeve top',
    1: 'long sleeve top',
    2: 'short sleeve outwear',
    3: 'long sleeve outwear',
    4: 'vest',
    5: 'sling',
    6: 'shorts',
    7: 'trousers',
    8: 'skirt',
    9: 'short sleeve dress',
    10: 'long sleeve dress',
    11: 'vest dress',
    12: 'sling dress'
}

frame_number = 0 

while ret:
    if frame_number == start_frame:
        results_detect = detect_model(frame)[0]

        for result in results_detect.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result

            if score > threshold:
                cropped_region = frame[int(y1):int(y2), int(x1):int(x2)]

                pattern_results = pattern_model(cropped_region)
                pattern_dict = pattern_results[0].names
                pattern_probs = pattern_results[0].probs.data.tolist()
                pattmax = pattern_dict[np.argmax(pattern_probs)]

                color_results = color_model(cropped_region)
                color_dict = color_results[0].names
                color_probs = color_results[0].probs.data.tolist()
                colormax = color_dict[np.argmax(color_probs)]

                label = f'{pattmax} {colormax} {results_detect.names[int(class_id)].upper()} '

               
                print(label)

    ret, frame = cap.read()
    frame_number += 1

   
    if frame_number > start_frame:
        break

cap.release()
cv2.destroyAllWindows()


AttributeError: 'NoneType' object has no attribute 'shape'